In [ ]:
import pandas as pd
import numpy as np
#from nphc.main import NPHC
#from nphc.utils.cumulants import Cumulants
from scipy.linalg import sqrtm, inv, eigh
from numpy import sqrt
import matplotlib
matplotlib.use('Agg')

import mlpp.hawkesnoparam as hnp

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

In [ ]:
def GetComponents(df, vol_bins):
    """ Given df representing one day of data, returns list of array representing 
        trades time for each volume component"""
    
    mask = lambda x,y: (df.OrderType==0).values * (np.abs(df.Volume.values)>x) * (np.abs(df.Volume.values)<=y)
    
    out = []
    for ii in range(len(vol_bins)-1): 
        
        out += [np.copy(df.loc[mask(vol_bins[ii], vol_bins[ii+1]), 'Time'].values)]
        
    return out

## Data store

In [ ]:
# data for Eurex futures are in this store
store = pd.HDFStore('/data/data/QH/L1.h5', 'r')

# each dataframe in the store represents one day
# name of the assets are 'xFDAX' and 'xFGBL'
# to access an asset on a particular day do
# store.get['_ast_name/YYYYmmdd']

In [ ]:
# to get one day
df = store.get('xFGBL/20140922')

## Non-parametric hawkes Bacry

In [ ]:
# in the paper we examined trades and considered multilpe components based on volume
# volume buckets:
bins4D = (0,1,3,10, np.inf)
bins6D = (0, 2, 3, 7, 20, np.inf)

### Estim conditional law

In [ ]:
# to do non par est we need to create estim object

# estimation parameters 
est_par = dict(hMax=2000, # max lag
               hMin=5e-4, # after hMin buckets are log-spaced
               hDelta=0.01, # regulates number of bins
               claw_method="log", n_threads=-1)

est = hnp.Estim(**est_par)

# do estimation using one month of data

asset = 'xFGBL'
days = pd.bdate_range(start='20140101', end='20140531')
big_data = []

# add realization and estimate claw
# 4D case
for d in days:
    
    try:
        df = store.get('%s/%s' %(asset, d.strftime('%Y%m%d')))
    except KeyError:
        continue
        
    data = GetComponents(df, bins4D)
    big_data.append(data)
    
    est.add_realization(data)

In [ ]:
# estimate conditional law values are in 
# est.claw (single index) or est.claw1[i][j]

# the corresponding x-values are in 
# est.claw_X

plt.plot(est.claw_X, est.claw1[0][0])
plt.loglog()

### Compute kernels

In [ ]:
# to compute the kernel do 
est.compute(n_quad=80, xmax=1, method='log', xmin=5e-4)

In [ ]:
# to access the kernels
# phi_x, phi_y = est.phi[i][j] 
phi_x, phi_y = est.phi[0][0]

plt.plot(phi_x, phi_y)
plt.loglog()

### Norms

In [ ]:
# norms are in 
# est.norm
sns.heatmap(est.norm, cmap='Blues')

# Retrieve the norm via NPHC

In [ ]:
len(big_data)

In [ ]:
from nphc.main import NPHC, starting_point
import nphc.utils.cumulants as cu

#cumul = cu.Cumulants(big_data)
list_cumul = [ cu.Cumulants(daily_data) for daily_data in big_data ]

In [ ]:
for cumul in list_cumul:
    cumul.set_all(H=1.)

In [ ]:
R0 = np.mean([starting_point(cumul) for cumul in list_cumul], axis=0)
#R0 = starting_point(list_cumulants)

In [ ]:
learning_rate=5e-2
training_epochs=3001
display_step=100

In [ ]:
R = NPHC(list_cumul,R0,alpha=0.,training_epochs=training_epochs,\
         display_step=display_step,learning_rate=learning_rate,optimizer='adam')

In [ ]:
d = list_cumul[0].dim
from scipy.linalg import inv
G = np.eye(d) - inv(R)
G0 = np.eye(d) - inv(R0)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, axs = plt.subplots(1, 2, sharey=True,figsize=(15,5))
labels = ['G0', 'G']

for ax, data, label in zip(axs, [G0, G], labels):
    
    
    colors = [plt.cm.viridis, plt.cm.magma, plt.cm.inferno, plt.cm.Blues, plt.cm.jet]
    color = colors[3]
    heatmap = ax.pcolor(data[::-1], cmap=color, alpha=.8, vmin=-0.1,vmax=.8)
        
    ax.set_xlabel(label,fontsize=30)
    plt.setp(ax.get_xticklabels(),visible=False)
    plt.setp(ax.get_yticklabels(),visible=False)
    
    fig.subplots_adjust(right=0.8)
    cax = fig.add_axes([0.82, 0.12, 0.02, .78])
    fig.colorbar(heatmap, cax=cax)